# AUTOMAÇÃO DE UPDATE 
##(Operational policy)

In [0]:
# ### Instalação de bibliotecas
# ##---------------------------

# !pip install gitlab
# !pip install python-gitlab
# !pip install jira

In [0]:
# Bibliotecas
import mysql.connector
import numpy as np
import pandas as pd
import requests
import gitlab
import base64
import json
import datetime
from jira import JIRA
from datetime import datetime

In [0]:
### Configura conexão com o banco de dados ReportDB
#-----------------------------------------------------------------------------

# Configurações
host = dbutils.secrets.get(DB_Host)
port = (DB_Port)
user = dbutils.secrets.get(DB_User)
password = dbutils.secrets.get(DB_Password)
database = dbutils.secrets.get(DB)
table_name = 'table_name'

# Criar a conexão com o banco de dados MySQL
conn = mysql.connector.connect(host=host, port=port, user=user, password=password, database=database)
cursor = conn.cursor()

# Executar a consulta SQL para obter os dados da tabela
query = f"SELECT * FROM {table_name}"
cursor.execute(query)

# Obter os nomes das colunas do banco de dados
columns = [col[0] for col in cursor.description]

# Recuperar os resultados da consulta
results = cursor.fetchall()

# Criar um DataFrame Pandas com os nomes das colunas corretos
df_operational_policy = pd.DataFrame(results, columns=columns)

# Fechar o cursor e a conexão
cursor.close()
conn.close()

# Exibir o DataFrame Pandas
display(df_operational_policy.head())

In [0]:
display(df_operational_policy.tail())

## Informações guardadas do banco

In [0]:
tipos_de_dados = df_operational_policy.dtypes
print(tipos_de_dados)

In [0]:
# Biblioteca com DePara para os nomes das colunas dentro do DF operational_policy
colunas = {'processo' : 'txt_process_number', 
           'emissao' : 'dt_emission', 
           'numero da apolice' : 'txt_policy_number',
           'razao_social' : 'txt_complainant_name',
           'razao_social_cnpj' : 'txt_complainant_cnpj_cfp',
           'segurado' : 'id_tribunal',
           'seguradora' : 'id_insurance_company'}

print(colunas)

In [0]:
# Lista com colunas tipo INT e FLOAT
coluna_int_float = []
coluna_datetime = []

tipos_de_dados = df_operational_policy.dtypes
tipos_de_dados = tipos_de_dados.to_dict()
for coluna, tipo_dado in tipos_de_dados.items():
  if tipo_dado == 'int' or tipo_dado == 'float':
    coluna_int_float.append(coluna)
  elif tipo_dado == 'datetime64[ns]' or coluna == 'dt_emission':
    coluna_datetime.append(coluna)

print(coluna_int_float)
print(coluna_datetime)

## Leitura do chamado

In [0]:
### Configura conexão com o Jira
#-----------------------------------------------------------------------------


# Construct a Client Instance
jiraOptions = {'server': "server.atlassian.com"}
email_id = "email_id"
token = "token_Jira"

# Pass the Authentication Parameter
jira = JIRA(options=jiraOptions, basic_auth=(email_id, token))

# Read the issue
card = dbutils.widgets.get("Chamado") #Como o script foi feito no databricks, podemos usar widgets para receber o card
singleIssue = jira.issue(card)
print('{}: {}:{}:{}'.format(singleIssue.key,
                         singleIssue.fields.summary,
                         singleIssue.fields.description,
                         singleIssue.fields.reporter.displayName))

comentario = singleIssue.fields.summary
print(comentario)

chave_chamado = singleIssue.key
print(chave_chamado)

chamado = singleIssue.fields.description
print(chamado)

## Informações para o Update

In [0]:
# Separando as palavras do chamado
chamado_palavras = chamado.split()
print(chamado_palavras)

In [0]:
# Pegando coluna a ser alterada no update
coluna_update = []

for palavra in chamado_palavras:
  if palavra in colunas:
    coluna_update.append(colunas[palavra])

print(f'A coluna a ser atualizada é: {coluna_update}')

In [0]:
# Pegando ID do update
id_update = []

for i, palavra in enumerate(chamado_palavras):
  if palavra == 'ID':
    id_index = i + 1
    id_update.append(chamado_palavras[id_index])

print(f'O ID a ser atualizado é: {id_update}')

In [0]:
# Pega o antigo valor para o Rollback
antigo_valor_update = []

for i, id in enumerate(id_update):
    antigo_valor_update.append(df_operational_policy.loc[int(id), coluna_update[i]])


for i, valor in enumerate(antigo_valor_update):
    if coluna_update[i] in coluna_datetime:
        dt_format_valor = "%Y-%m-%d 00:00:00"
        if isinstance(valor, datetime):
            antigo_valor_update[i] = valor.strftime(dt_format_valor)

print(antigo_valor_update)

In [0]:
# Pegando o novo valor para realizar o Update
novo_valor = []

for i, palavra in enumerate(chamado_palavras):
  if palavra == 'novo_valor':
    id_index_novo_valor = i + 1
    novo_valor.append(chamado_palavras[id_index_novo_valor])


for i, valor in enumerate(novo_valor):
  if coluna_update[i] in coluna_datetime and len(novo_valor[i]) < 20:
    novo_valor[i] = novo_valor[i] + " 00:00:00"

print(f'O novo valor é: {novo_valor}')

## Script do Update

In [0]:
# Informações de data e hora do changeset
agora = datetime.now()
formato = "%Y-%m-%d-%H%M"
data_hora_formatada = agora.strftime(formato)

data_hora_formatada

In [0]:
# Changeset
update_head =f'''-- liquibase formatted sql
-- changeset Nome Sobrenome:{data_hora_formatada}
-- comment [{chave_chamado}] {comentario}'''

print(update_head)

In [0]:
#### QUERY UPDATE

# Variáveis
indice = 0
updates = {}

# Montagem
for i, id in enumerate(id_update):
  if coluna_update[i] in coluna_int_float:
    if antigo_valor_update[i]:
      updates[indice+1] = f'''update operational_policy set {coluna_update[i]} = {novo_valor[i]} where id = {id_update[i]};
-- rollback update operational_policy set {coluna_update[i]} = {antigo_valor_update[i]} where id = {id_update[i]};\n\n'''
      indicie += 1
    else:
      updates[indice+1] = f'''update operational_policy set {coluna_update[i]} = {novo_valor[i]} where id = {id_update[i]};
-- rollback update operational_policy set {coluna_update[i]} = NULL where id = {id_update[i]};\n\n'''
      indicie += 1
  else:
    if antigo_valor_update[i]:
      updates[indice+1] = f'''update operational_policy set {coluna_update[i]} = \'{novo_valor[i]}\' where id = {id_update[i]};
-- rollback update operational_policy set {coluna_update[i]} = \'{antigo_valor_update[i]}\' where id = {id_update[i]};\n\n'''
      indice += 1
    else:
      updates[indice+1] = f'''update operational_policy set {coluna_update[i]} = \'{novo_valor[i]}\' where id = {id_update[i]};
-- rollback update operational_policy set {coluna_update[i]} = NULL where id = {id_update[i]};\n\n'''
      indice += 1
    
print(updates)

In [0]:
# Monta a query para o arquivo do commit
query = ''
for valores in updates.values():
  if valores not in query:
    query += valores
    
query_completa = update_head + '\n\n' + query[:-2]
print(query_completa)

##Conexão com o Gitlab

In [0]:
# Tratando branch name
tratando_caracteres = {'ç' : 'c', 'á' : 'a', 'à' : 'a', 'â' : 'a', 'ã' : 'a', 'é' : 'e', 'ê' : 'e', 'í' : 'i', 'ó' : 'o', 'ô' : 'o', 'õ' : 'o', 'ò' : 'o', ' ' : '-', 'ú' : 'u'}

commit_message = singleIssue.fields.summary
commit_message = commit_message.lower()
print(commit_message)

branch_name_comment = singleIssue.fields.summary
branch_name_comment = branch_name_comment.lower()
for caracter, letra in tratando_caracteres.items():
  if caracter in branch_name_comment:
    branch_name_comment = branch_name_comment.replace(caracter, letra)

print(branch_name_comment)

In [0]:
### Configura conexão com o Gitlab
#-----------------------------------------------------------------------------


# Configurações
gitlab_url = r"https://gitlab.com"
support_project_id = "project_id" 
gitlab_token = "token_acesso_gitlab"

# Crie uma instância da classe Gitlab
gl = gitlab.Gitlab(gitlab_url, private_token=gitlab_token)

# Autentique-se no GitLab
gl.auth()

In [0]:
# Lendo o projeto em que fazemos update
project = gl.projects.get(support_project_id)
print(f"Informações sobre o projeto {project.name}:")
print(f"Descrição: {project.description}")
print(f"URL do repositório: {project.web_url}")

# Lendo branches do projeto
branch_main = project.branches.list()
print(branch_main)

In [0]:
# Criando nova branch
branch_name = f'hotfix/{chave_chamado}-{branch_name_comment}'

branch = project.branches.create({'branch': branch_name,
                                  'ref': 'main'})

In [0]:
# Novo arquivo
arquivo_changes = f'arquivo_nome/{data_hora_formatada}-{branch_name_comment}.sql'

# Conteúdo do commit
query_update = query_completa

# Fazendo commit dentro da branch
data = {
    'branch': branch_name,
    'commit_message': f'[{chave_chamado}] {commit_message}',
    'actions': [
        {
            'action': 'create',
            'file_path': arquivo_changes,
            'content': query_update,
        }
    ]
}

commit = project.commits.create(data)

In [0]:
# Lista os merges criados por mim (Dono do token)
mrs = gl.mergerequests.list()
print(mrs)

In [0]:
print(branch_name)

In [0]:
# ## Deletando branch
# # Rodar segundo a necessidade

# project.branches.delete(branch_name)
# print(f"Branch {branch_name} deletada")